In [1]:
import numpy as np

from itertools import product

##### Exercise 5.1

Consider the diagrams on the right in Figure  5.2. Why does the value function jump up for the last two rows in the rear? Why does it drop off for the whole last row on the left? Why are the frontmost values higher in the upper diagrams than in the lower?

Jumps up for last two rows in the rear since sticking for 20 and 21 in blackjack usually result in a win. It drops off for the last row on the left since if the dealer has an Ace, it's bad news for the player. Finally, the frontmost values are higher in the upper diagrams than in the lower since having a usable Ace makes it less likely that an Ace will make the player bust.

##### Exercise 5.2

The backup diagram for the Monte Carlo estimation of $Q^\pi$ is similar to the backup diagram for $V^\pi$, but the root is a state-action pair, not just a state. The diagram ends in a terminal state.

##### Some notes 
"Without a model (as we had in DP chapter 4)... state values alone are not sufficient. One must explicitly estimate the value of each action in order for the values to be useful in suggesting a policy. "

By model, the author means, having the transition probabilities and transition rewards available at hand.

" For policy evaluation to work for action values, we must assure continual exploration. One way to do this is by specifying that the first step of each episode starts at a state-action pair, and that every such pair has a nonzero probability of being selected as the start. This guarantees that all state-action pairs will be visited an infinite number of times in the limit of an infinite number of episodes. We call this the assumption of exploring starts."


Has the following been proved on Monte Carlo ES?
"Convergence to this optimal fixed point seems inevitable as the changes to the action-value function decrease over time, but has not yet been formally proved. In our opinion, this is one of the most fundamental open questions in reinforcement learning."

##### Questions

**Question**: In 5.6, in the Figure 5.7, (c), the update to w, why is there a 1 in the numerator? It's because $\pi(s,a)$ is a deterministic policy!

Why are we taking $\tau$ to be the latest time at which the actions are not greedy? Is it because our estimate for $Q^\pi$ only improves for nongreedy actions as stated in the section?

**Question**: In 5.4, the conditions for the policy improvement theorem require optimal substructure right? So even though Monte Carlo is more robust to violations of the Markov Property since it doesn't bootstrap $V$ or $Q$, we are still assuming that greedy updates in GPI (Generalized Policy Improvement) will allow us to arrive at the optimal action-value functions due to the Markov Property?

##### Exercise 5.3 

What is the Monte Carlo estimate analogous to (5.3) for action values, given returns generated using $\pi'$?

I'm not sure, but I think it would be something like:

$$Q(s,a) = \frac{\sum_i^{n_s} \frac{p_i(s,a)}{p_i'(s,a)} R_i(s,a) }{ \sum_i^{n_s} \frac{p_i(s,a)}{p_i'(s,a)} }$$

Where $R_i(s,a)$ is the reward following state $s$ and action $a$, and $p_i(s_t, a_t) = \prod_{k=t}^{T_i(s) - 2} \pi(s_k, a_k)P^a_{s_k, s_{k+1}} \cdot \pi(s_{T_i(s) - 1}, a_{T_i(s) - 1}) $. 



##### Exercise 5.4

In [297]:
class Track(object):
    def __init__(self):
        """
            0 = off track
            1 = road - on track
            2 = start line
            3 = finish line
        """
        self.track = np.ones((18, 18))
        self.track[5:18, 5:18] = 0
        self.track[-1, :] *= 2
        self.track[:, -1] *= 3

    def get_next_position(self, racecar):
        """
            RaceCar racecar: RaceCar object
        """
        
        reward = -1
        
#         print('starting position is ', (racecar.x, racecar.y))
#         print('starting velocity is ', (racecar.velocity_x, racecar.velocity_y))
        
        new_x = racecar.x + racecar.velocity_x
        new_y = racecar.y + racecar.velocity_y
        
        final_x = new_x
        final_y = new_y
        
        # Compute all the unique boxes we hit on a line between the start and end points
        x_positions = np.linspace(racecar.x, new_x, num=20)
        y_positions = np.linspace(racecar.y, new_y, num=20)
        positions = zip(x_positions, y_positions)
        positions = [(np.floor(x), np.floor(y)) for x, y in positions]
        
        # Get unique discrete positions visited during this time step
        ordered_positions = []
        for pos in positions:
            if len(ordered_positions) == 0 or pos != ordered_positions[-1]:
                ordered_positions.append(pos)
                        
        # Check if the car crashes into the track at any of those time points
        for pos_idx, pos in enumerate(ordered_positions):
            
            # speed past the finish without penalty?
            if self.is_terminal_state_from_coordinates(pos[0], pos[1]):
                reward = -1
                final_x, final_y = ordered_positions[pos_idx]
                break

            if self.is_out_of_bounds(pos):
#                 print('CRASH')
                reward = -5
                crash_x, crash_y = pos
                final_x, final_y = ordered_positions[pos_idx - 1]
                racecar.velocity_x = 0
                racecar.velocity_y = 0
                break

        # If the car is not moving, move at least 1 step
        if final_x == racecar.x and final_y == racecar.y:
            if self.is_out_of_bounds((final_x + 1, final_y)):
                final_y += 1
                racecar.velocity_y = 1
            elif self.is_out_of_bounds((final_x, final_y + 1)):
                final_x += 1
                racecar.velocity_x = 1
            else:
                random_choice = np.random.choice([0, 1])
                final_x += random_choice
                final_y += (1 - random_choice)
                racecar.velocity_x += random_choice
                racecar.velocity_y += (1 - random_choice)                    
        
#         print('update position is ', (final_x, final_y))
        
        racecar.x = final_x
        racecar.y = final_y
        
        return reward

    def convert_cartesian_to_indexes(self, x, y):
        y_prime, x_prime = x, y
        x_prime = self.track.shape[0] - x_prime - 1
        return int(x_prime), int(y_prime)
    
    def convert_indexes_to_cartesian(self, x, y):
        y_prime, x_prime = x, y
        y_prime = self.track.shape[1] - y_prime - 1
        return int(x_prime), int(y_prime)
    
    def is_terminal_state(self, racecar):
        x, y = self.convert_cartesian_to_indexes(racecar.x, racecar.y)
        if self.track[x, y] == 3:
            return True
        return False
    
    def is_terminal_state_from_coordinates(self, x, y):
        x, y = self.convert_cartesian_to_indexes(x, y)
        if self.track[x, y] == 3:
            return True
        return False
    
    def is_out_of_bounds(self, position):
        x, y = position
        
        if x < 0 or x >= self.track.shape[1]:
            return True
        
        if y < 0 or y >= self.track.shape[0]:
            return True

        # y is reversed in our frame of reference
        x, y = self.convert_cartesian_to_indexes(x, y)

        if self.track[x, y] == 0:
            return True
        
        return False
    
    def get_random_start(self):
        # returns x and y coordinates of random start
        starts = np.argwhere(self.track == 2)
        random_start = np.random.randint(len(starts))
        start = starts[random_start]
        return self.convert_indexes_to_cartesian(*start)
    
    def get_states(self):
        return [self.convert_indexes_to_cartesian(x, y) for x, y in np.argwhere(self.track != 0)]
    
    def print_track(self, x, y):
        x, y = self.convert_cartesian_to_indexes(x, y)
        pt = np.copy(self.track)
        pt[x, y] = -1
        print(pt)
        
    

In [326]:
class RaceCar(object):
    def __init__(self):
        self.velocity_x = 0
        self.velocity_y = 0
        self.x = 0
        self.y = 0
        
        self.MAX_VELOCITY = 5
        self.MIN_VELOCITY = 0

    def get_episode(self, pi, track, actions, states, greedy=False, verbose=False):
        """
            actions: an index to action dictionary
        
        """
        self.velocity_x = 0; self.velocity_y = 0
        self.x, self.y = track.get_random_start()
        rewards = []
        visited_states = [((self.x, self.y), (self.velocity_x, self.velocity_y))]
        saved_actions = []
        
        terminated = False
        while not terminated:
            state_idx = states[((self.x, self.y), (self.velocity_x, self.velocity_y))]

            # Choose action according to PI
            if greedy:
                action_idx = np.where(pi[state_idx, :] == np.amax(pi[state_idx, :]))[0]
                action_idx = np.random.choice(action_idx)   
            else:
                action_idx = np.random.choice(len(actions), size=1, p=pi[state_idx, :])[0]    
            
            action = actions[action_idx]
            saved_actions.append(action)
            
            # Take the action
            self.velocity_x += action[0]
            self.velocity_y += action[1]

            self.velocity_x = min(max(self.velocity_x, self.MIN_VELOCITY), self.MAX_VELOCITY)
            self.velocity_y = min(max(self.velocity_y, self.MIN_VELOCITY), self.MAX_VELOCITY)

            # save the rewards and states
            reward = track.get_next_position(self)
            if len(visited_states) > 100:
                reward = -1000
                terminated = True
            else:
                terminated = track.is_terminal_state(self)
            
            rewards.append(reward)
            visited_states.append(((self.x, self.y), (self.velocity_x, self.velocity_y)))
            
            if verbose:
                track.print_track(self.x, self.y)
                print('Velocity is now: ', (self.velocity_x, self.velocity_y))
        
        return visited_states, saved_actions, rewards
        
    def get_velocity_states(self):
        return list(product(
                range(self.MIN_VELOCITY, self.MAX_VELOCITY + 1),
                range(self.MIN_VELOCITY, self.MAX_VELOCITY + 1)
            )
        )

In [327]:
car = RaceCar()
track = Track()


In [328]:
# On Policy Monte Carlo for the Race Track problem
actions_list = list(product([-1, 0, 1], [-1, 0, 1]))
actions_to_idx = {action: idx for idx, action in enumerate(actions_list)}
idx_to_actions = {idx: action for idx, action in enumerate(actions_list)}

states_list = list(product(track.get_states(), car.get_velocity_states()))
states_to_idx = {state: idx for idx, state in enumerate(states_list)}

Q = np.random.random((len(states_to_idx), len(actions_to_idx)))
Returns = {(s, a): [] for s, a in product(states_to_idx, actions_to_idx)}

pi = np.random.random((len(states_to_idx), len(actions_to_idx)))
pi = pi / np.sum(pi, axis=1)[:, None]

In [333]:
# On Policy Monte Carlo

count = 0
iamhappy = True
epsilon = 0.1
    
while iamhappy:

    visited_states, actions_taken, rewards = car.get_episode(pi, track, idx_to_actions, states_to_idx)
    
    has_visited_first_occurence = {}
    for idx, sa in enumerate(zip(visited_states, actions_taken)):
        s, a = sa
        if (s, a) not in has_visited_first_occurence:
            Returns[(s, a)].append(sum(rewards[idx:]))
            Q[states_to_idx[s], actions_to_idx[a]] = np.mean(Returns[(s, a)]) 
            has_visited_first_occurence[(s, a)] = 0
    
    for s in visited_states:
        # We can take the greedy action, but it's probably better to break ties
        # a_star = np.argmax(Q[states_to_idx[s],:])
        action_idx = np.where(Q[states_to_idx[s],:] == np.amax(Q[states_to_idx[s],:]))[0]
        a_star = np.random.choice(action_idx)
        for action_idx, a in enumerate(actions_list):
            if a_star == action_idx:
                pi[states_to_idx[s], action_idx] = 1 - epsilon + epsilon / len(actions_list)
            else:
                pi[states_to_idx[s], action_idx] = epsilon / len(actions_list)
            
    count += 1
    
    if count >= 4000: iamhappy = False

In [334]:
car.get_episode(pi, track, idx_to_actions, states_to_idx, greedy=True, verbose=True)

[[ 1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  3.]
 [ 1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  3.]
 [ 1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  3.]
 [ 1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  3.]
 [ 1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  3.]
 [ 1.  1.  1.  1.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 1.  1.  1.  1.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 1.  1.  1.  1.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 1.  1.  1.  1.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 1.  1.  1.  1.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 1.  1.  1.  1.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 1.  1.  1.  1.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 1.  1.  1.  1.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 1.  1.  1.  1.  1.  0.

([((1, 0), (0, 0)),
  ((1, 1), (0, 1)),
  ((1, 2), (0, 1)),
  ((1, 4), (0, 2)),
  ((1, 7), (0, 3)),
  ((2, 9), (1, 2)),
  ((4, 12), (2, 3)),
  ((5, 14), (1, 2)),
  ((7, 16), (2, 2)),
  ((10, 17), (3, 1)),
  ((14, 17), (4, 0)),
  ((17.0, 17.0), (4, 0))],
 [(-1, 1),
  (-1, 0),
  (0, 1),
  (-1, 1),
  (1, -1),
  (1, 1),
  (-1, -1),
  (1, 0),
  (1, -1),
  (1, -1),
  (0, -1)],
 [-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1])

In [146]:
# class MonteCarloSolver(object):
#     def __init__(self, actions, states, agent, environment):
#         self.Q = np.random.random((len(states), len(actions)))
#         self.Returns = np.zeros((len(states), len(actions)))

#         self.pi = np.random.random((len(states), len(actions)))
#         self.pi = self.pi / np.sum(self.pi, axis=1)[:, None]
        
#         self.agent = agent
#         self.environment = environment

#     def solve(self):
#         pass

##### Exercise 5.5

Modify the algorithm for first-visit MC policy evaluation (Figure  5.1) to use the incremental implementation for stationary averages described in Section 2.5.

We just need to update the algorithm so that $Returns(s)$ is a 1x1 array for each $s \in S$. Before part(b) of the algorithm, we need to initialize $k = 0$, and in part (b) in the loop, we do:

- $Returns(s) \leftarrow Returns(s) + \frac{1}{k + 1}[R - Returns(s)]$ 
- $k \leftarrow k + 1$


##### Exercise 5.6


Derive the weighted-average update rule (5.5) from (5.4). Follow the pattern of the derivation of the unweighted rule (2.4) from (2.1).

We have, 

$$V_{n+1} = \frac{\sum_{k=1}^{n+1} w_k R_k }{\sum{k+1}{n+1} w_k}$$
$$= \frac{w_{n+1} R_{n+1} + \sum_{k=1}^n w_k R_k }{W_{n+1}}$$

where $W_{n+1} = W_n + w_{n+1}$ and $W_0 = 0$. Then we have:

$$V_{n+1} = \frac{1}{W_{n+1}} [w_{n+1}R_{n+1} + V_n W_n] $$
$$ = \frac{1}{W_{n+1}} [w_{n+1}R_{n+1} + V_n (W_{n+1} - w_{n+1})] $$
$$ = V_n +  \frac{w_{n+1}}{W_{n+1}} [R_{n+1} - V_n ] .$$


##### Exercise 5.7

Modify the algorithm for the off-policy Monte Carlo control algorithm (Figure  5.7) to use the method described above for incrementally computing weighted averages.

Before the repeat loop, we need to initialize $W = 0$. Then in the repeat loop we do:

- get $w$ and $t$ as usual
- delete lines for $N(s,a)$ and $D(s,a)$
- $W \leftarrow w + W$
- $Q(s,a) \leftarrow Q(s,a) + \frac{w}{W} [R_t - Q(s,a)] $